In [ ]:
from IPython.core.display import HTML,display
import pandas as pd
from numpy import full
def final_sm(data_array, row_labels,col_labels):
    """Show an HTML table from a 2d numpy array"""
    df = pd.DataFrame(data_array,index=row_labels,columns=col_labels)
    table_html = df.to_html()
    return HTML(table_html)

In [ ]:
#lab 1 = Smith and Waterman
#build an array of zeroes
seq1 = input("Enter sequence 1: ")
seq2 = input("Enter sequence 2: ")
n_rows = len(seq2) + 1 #need an extra row up top
n_columns = len(seq1) + 1 #need an extra column on the left
row_labels = [label for label in "-"+seq2]
column_labels = [label for label in "-"+seq1]
scoring_array = full([n_rows,n_columns],0)
traceback_array = full([n_rows,n_columns],"-")
print(scoring_array)
print(traceback_array)
#Define Unicode arrows we'll use in the traceback array
up_arrow = "\u2191"
left_arrow = "\u2190"
up_left_arrow = "\u2196"
arrow = "-"
gap_penalty = -4 #w
match_bonus = 5
mismatch_penalty = -3
#iterate over columns first because we want to do
# all the columns for row 1 before row 2
for row in range(n_rows):#o/p of range will be [0,1,2,3,4] if seq="AGCC"
    for col in range(n_columns):#o/p of range will be [0,1,2,3,4,5] if seq="ATGCC"
        if row == 0 and col == 0:
            #We're in the upper left corner
            score = 0
            arrow = "-"
        elif row == 0:
            #We're on the first row, but NOT in the corner
            #Look up the score of the previous cell (to the left) in the score array\
            previous_score = scoring_array[row,col - 1]
            # add the gap penalty to it's score
            score = previous_score
            arrow = left_arrow
        elif col == 0:
            #We're on the first column but not in the first row
            previous_score = scoring_array[row -1,col]
            score = previous_score
            arrow = up_arrow
        else: #We're in a 'middle' cell of the alignment
            #Calculate the scores for coming from above,
            #from the left, (representing an insertion into seq1)
            cell_to_the_left = scoring_array[row,col-1]
            from_left_score = cell_to_the_left + gap_penalty
            #or from the above (representing an insertion into seq2)
            above_cell = scoring_array[row-1,col]
            from_above_score = above_cell + gap_penalty
            #diagonal cell, representing a matching (e.g. A --> A or G --> G etc.,)
            diagonal_left_cell = scoring_array[row-1,col-1]
            #NOTE: since the table has an extra row and column (the blank ones),
            #when indexing back to the sequence we want row -1 and col - 1.
            #since row 1 represents character 0 of the sequence.
            if seq1[col-1] == seq2[row-1]:
                diagonal_left_cell_score = diagonal_left_cell + match_bonus #i.e., 5
            else:
                diagonal_left_cell_score = diagonal_left_cell + mismatch_penalty #i.e.,-3
            score = max([from_left_score, from_above_score, diagonal_left_cell_score, 0])
            #take the max
            #make note of which cell was the max in the traceback array
            #using Unicode arrows
            if score == from_left_score:
                arrow = left_arrow
            elif score == from_above_score:
                arrow = up_arrow
            elif score == diagonal_left_cell_score:
                arrow = up_left_arrow
        traceback_array[row,col]=arrow
        scoring_array[row,col] = score
print(seq1)
print(seq2)
display(final_sm(scoring_array,row_labels,column_labels))
display(final_sm(traceback_array,row_labels,column_labels))
#traceback step
r = row
c = col
seq1_aln = ''
seq2_aln = ''
large =  r if r>c else c # r>c?r:c
for nt in range(large):
  if(traceback_array[r,c] ==  up_left_arrow):
    seq1_aln += seq1[c-1]
    seq2_aln += seq2[r-1]
    r = r - 1
    c = c - 1
  elif(traceback_array[r,c] ==  left_arrow):
    seq1_aln += seq1[c-1]
    seq2_aln += "-"
    c = c - 1
  elif(traceback_array[r,c] == up_arrow):
    seq2_aln += seq2[r-1]
    seq1_aln += "-"
    r = r - 1
#print alignmentA
alnsym = ''
print("\033[1;33;44m", seq1_aln[::-1])
for nt in range(large):
  alnsym += "|" if (seq1_aln[nt] == seq2_aln[nt]) else " "
print("\033[1;36;50m",alnsym[::-1])
print("\033[1;33;44m", seq2_aln[::-1])